In [1]:
# Import the necessary modules
import SciServer
from SciServer import SkyServer
import matplotlib.pyplot as plt
from astropy.io import ascii, fits
from astropy.table import Table
import numpy as np

/home/fortson/manth145/.conda/envs/tf_latest/lib/python3.9/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [2]:
data_file = fits.getdata('/home/fortson/manth145/data/Misc/LEAM/LEAM_sample.fits')

In [3]:
queried_table = Table(names=['ID', 'RA', 'DEC', 'ObjID', 'SpecObjID', 'logM', 'Z', 'Age', 'sSFR', 'halpha_flux'],
                     dtype=['str', 'float', 'float', 'int', 'int', 'float', 'float','float', 'float', 'float'])

In [5]:
len(data_file)

82

In [4]:
# If the info you are providing is a tuple of (RA, DEC) positions, then use the following. Load a catalog and pass a list of tuples to iteratively 
# fetch the images.
info_type ='ra_dec'

'''
If you have ObjIDs (unique to each data release), then comment the above and use the following.
'''
# info = [1237671932283322445]
# info_type ='ID'

obj_info_search_DR = "DR16" # Data release to be used to query the ObjID, PetroR90 information
image_data_release = "DR7" # Data release from which the Jpeg should be queried.


for each_info in data_file: #iterate over the loop

    ra, dec = (each_info['RA'], each_info['DEC']) #unpack the RA and DEC info

    object_search_data =  SciServer.SkyServer.objectSearch(ra= ra, dec = dec, dataRelease=obj_info_search_DR) # Query the object information
    object_id = object_search_data[0]['Rows'][0]['objId'] # grab the ObjID 
    spec_obj_id = object_search_data[0]['Rows'][0]['specObjId']
    if spec_obj_id !='' and object_id!= '':
    #     sql_query = "SELECT p.ra, p.dec, p.ObjID, p.PetroR90_r from PhotoPrimary p WHERE p.objID = %s"%(object_id) # set up SQL query to retrieve PetroR50_r
        spec_query = "SELECT g.h_alpha_flux from galSpecLine g WHERE g.specObjID = %s"%(spec_obj_id)
        stellar_mass_query = "SELECT m.logMass, m.metallicity, m.age, m.ssfr from stellarMassFSPSGranWideDust m WHERE m.specObjID = %s"%(spec_obj_id)
    #     queried_data = SciServer.SkyServer.sqlSearch(sql_query, dataRelease=obj_info_search_DR) # queried data
        try:
            spec_queried_data = SciServer.SkyServer.sqlSearch(spec_query, dataRelease=obj_info_search_DR).to_numpy()[0]
        except IndexError:
            spec_queried_data = np.array([-99])
        
        try:
            physical_properties = SciServer.SkyServer.sqlSearch(stellar_mass_query, dataRelease=obj_info_search_DR).to_numpy()[0]
        except IndexError:
            physical_properties = np.array([-99, -99, -99, -99])

        queried_table.add_row(np.concatenate([[each_info['ID']], [each_info['RA']], [each_info['DEC']],[object_id], [spec_obj_id], physical_properties, spec_queried_data]))
    else:
        if object_id == '':
            object_id = -99
        queried_table.add_row(np.concatenate([[each_info['ID']], [each_info['RA']], [each_info['DEC']],[object_id], [-99], [-99,-99,-99,-99], [-99]]))


/home/fortson/manth145/.conda/envs/tf_latest/lib/python3.9/site-packages/SciServer-2.1.0-py3.9.egg/SciServer/SkyServer.py:331: Warning: In Authentication.getToken: Authentication token is not defined: the user did not log in with the Authentication.login function, or the token has not been stored in the command line argument --ident.
/home/fortson/manth145/.conda/envs/tf_latest/lib/python3.9/site-packages/SciServer-2.1.0-py3.9.egg/SciServer/SkyServer.py:47: Warning: In Authentication.getToken: Authentication token is not defined: the user did not log in with the Authentication.login function, or the token has not been stored in the command line argument --ident.
/home/fortson/manth145/.conda/envs/tf_latest/lib/python3.9/site-packages/SciServer-2.1.0-py3.9.egg/SciServer/SkyServer.py:47: Warning: In Authentication.getToken: Authentication token is not defined: the user did not log in with the Authentication.login function, or the token has not been stored in the command line argument --i

In [8]:
object_id

''

In [6]:
ascii.write(queried_table, '/home/fortson/manth145/data/Misc/LEAM/Leam_Sample_SDSS_Info.csv', format='csv', overwrite=True)

In [5]:
queried_table

ID,RA,DEC,ObjID,SpecObjID,logM,Z,Age,sSFR,halpha_flux
str16,float64,float64,int64,int64,float64,float64,float64,float64,float64
dm1152+07a,178.12779166666667,7.117361111111111,1237661971182583838,1825092052086450176,8.99454263890357,0.0076999999582767,1.29934740066528,-0.282896220684052,66.54812
dm1152+07b,178.12525,7.1177777777777775,1237661971182583837,1826262757125679104,8.74649397211624,0.0120000001043081,1.29934740066528,-0.285143464803696,1625.038
dm0035-10a,8.951291666666666,-9.209972222222222,1237652948529512701,-99,-99.0,-99.0,-99.0,-99.0,-99.0
dm0035-10b,8.951166666666667,-9.205527777777778,1237652948529512701,-99,-99.0,-99.0,-99.0,-99.0,-99.0
dm1059+47a,164.77733333333333,47.463,1237658611979583536,1085448093815564288,9.27777067881912,0.0120000001043081,2.90695571899414,-0.699218332767487,114.7458
dm1059+47b,164.7697083333333,47.46344444444444,1237658611979583535,1084249351213049856,9.10080710749947,0.0076999999582767,1.50865399837494,-0.521150290966034,421.3619
dm1142+52a,175.754375,52.465333333333334,1237657857145700638,990933519855282176,8.85216033225342,0.0076999999582767,1.30688512325287,-0.29055792093277,32.60795
dm1142+52b,175.739125,52.465555555555554,1237657857145634895,992015439313790976,9.83284797619942,0.0149999996647239,4.49568796157837,-1.04490375518799,1206.64
dm1403+41a,210.80729166666666,41.6095,1237661872411902003,1516684803536087040,9.29331144230432,0.0120000001043081,1.29934740066528,-0.285143464803696,782.7853


In [8]:
object_search_data[0]['Rows'][0]

{'objId': '1237652948529512701',
 'specObjId': '',
 'apid': '',
 'id': 1237652948529512701,
 'specId': None,
 'name': ''}

In [20]:
np.array(spec_queried_data)

array([[66.54812 ,  2.522923]])

In [22]:
spec_queried_data['h_alpha_flux'][0]

66.54812

In [6]:
physical_properties.to_numpy()[0]

AttributeError: 'numpy.ndarray' object has no attribute 'to_numpy'

In [27]:
physical_properties

,logMass,metallicity,age,ssfr
0,9.481586,0.0077,6.58356,-1.096489


In [17]:
spec_queried_data

array([66.54812])

In [16]:
physical_properties

array([ 9.48158557e+00,  7.69999996e-03,  6.58355951e+00, -1.09648907e+00])

In [25]:
np.concatenate([[each_info['ID']], [each_info['RA']], [each_info['DEC']],[object_id], [spec_obj_id], physical_properties, spec_queried_data])

array(['dm1152+07a', '178.12779166666667', '7.117361111111111',
       '1237661971182583838', '1825092052086450176', '9.48158557056236',
       '0.0076999999582767', '6.58355951309204', '-1.09648907184601',
       '66.54812'], dtype='<U32')

In [19]:
physical_properties.ravel()

array([ 9.48158557e+00,  7.69999996e-03,  6.58355951e+00, -1.09648907e+00])

In [27]:
queried_table

ID,RA,DEC,ObjID,SpecObjID,logM,Z,Age,sSFR,halpha_flux
str10,float64,float64,int64,int64,float64,float64,float64,float64,float64
dm1152+07a,178.12779166666667,7.117361111111111,1237661971182583838,1825092052086450176,9.48158557056236,0.0076999999582767,6.58355951309204,-1.09648907184601,66.54812
